In [1]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
from scipy import stats
import numpy as np
import sys

def weeknum(d):
	return d.weekday()

def isocalendar(d):
	return d.isocalendar()

In [2]:
OUTPUT_TEMPLATE = (
    "Initial (invalid) T-test p-value: {initial_ttest_p:.3g}\n"
    "Original data normality p-values: {initial_weekday_normality_p:.3g} {initial_weekend_normality_p:.3g}\n"
    "Original data equal-variance p-value: {initial_levene_p:.3g}\n"
    "Transformed data normality p-values: {transformed_weekday_normality_p:.3g} {transformed_weekend_normality_p:.3g}\n"
    "Transformed data equal-variance p-value: {transformed_levene_p:.3g}\n"
    "Weekly data normality p-values: {weekly_weekday_normality_p:.3g} {weekly_weekend_normality_p:.3g}\n"
    "Weekly data equal-variance p-value: {weekly_levene_p:.3g}\n"
    "Weekly T-test p-value: {weekly_ttest_p:.3g}\n"
    "Mann–Whitney U-test p-value: {utest_p:.3g}"
)


In [3]:
counts = pd.read_json('reddit-counts.json.gz', lines=True)
counts = counts[counts['subreddit'] == 'canada']
counts['week_num'] = counts['date'].apply(weeknum)

In [4]:
data_2012 = counts[counts['date'].dt.year == 2012]
data_2013 = counts[counts['date'].dt.year == 2013]
counts = pd.concat([data_2012, data_2013], axis = 0, ignore_index = True)
    #https://jingyan.baidu.com/article/91f5db1b79205a1c7f05e3ae.html

In [5]:
data_Sat = counts[counts['week_num'] == 5]
data_Sun = counts[counts['week_num'] == 6]

In [6]:
data_weekend = pd.concat([data_Sat, data_Sun], axis=0, ignore_index=True)
    #https://jingyan.baidu.com/article/91f5db1b79205a1c7f05e3ae.html

In [7]:
data_weekday = counts[counts['week_num'] != 5]
data_weekday = data_weekday[data_weekday['week_num'] != 6]

In [8]:
initial_Ttest = stats.ttest_ind(data_weekend['comment_count'], data_weekday['comment_count']).pvalue

In [9]:
normality_pvalue_weekday = stats.normaltest(data_weekday['comment_count']).pvalue
normality_pvalue_weekend = stats.normaltest(data_weekend['comment_count']).pvalue

In [10]:
levene_pvalue= stats.levene(data_weekend['comment_count'], data_weekday['comment_count']).pvalue

In [11]:
data_weekend['root'] = data_weekend['comment_count'].apply(np.sqrt) #good
data_weekday['root'] = data_weekday['comment_count'].apply(np.sqrt) #bad
out5 = stats.normaltest(data_weekday['root']).pvalue
out6 = stats.normaltest(data_weekend['root']).pvalue 
out7 = stats.levene(data_weekend['root'], data_weekday['root']).pvalue #good

del data_weekend['root']
del data_weekday['root']

In [19]:
data_weekend['iso'] = data_weekend['date'].apply(isocalendar)
data_weekend['year']= data_weekend['iso'].apply(pd.Series).astype(int)[0]
data_weekend['week'] = data_weekend['iso'].apply(pd.Series).astype(int)[1]

del data_weekend['iso']
data_weekend_iso = data_weekend.groupby(['year','week']).mean()
del data_weekend_iso['week_num']

In [20]:
data_weekend_iso

comment_count
year week               
2011 52            995.0
2012 1            1163.0
     2            1372.0
     3             915.5
     4            1285.0
     5            1228.0
     6             980.5
     7            1273.5
     8            1336.5
     9            1021.0
     10           1353.0
     11           1282.0
     12           1759.0
     13           1481.0
     14           1368.5
     15           1423.5
     16           1586.0
     17           1623.5
     18           1051.5
     19           1275.5
     20           1196.5
     21           1311.0
     22           1728.5
     23           1725.0
     24           1109.0
     25           1222.0
     26           1173.0
     27            982.0
     28           1272.0
     29           1761.5
...                  ...
2013 23            933.0
     24            882.0
     25            895.0
     26            844.0
     27            965.0
     28           1018.5
     29            761.0
     30           1071.5
     31           1403.0
     32           1759.5
     33           1335.0
     34           1229.0
     35           1349.0
     36           1508.0
     37           1286.0
     38           1055.5
     39           1238.5
     40           1096.5
     41           1108.0
     42           1986.5
     43           1349.0
     44           1371.5
     45           1318.5
     46           1368.5
     47           1363.0
     48           1553.0
     49           1412.5
     50           1547.0
     51           1173.5
     52           1117.5

[105 rows x 1 columns]

In [17]:
data_weekday['iso'] = data_weekday['date'].apply(isocalendar)
data_weekday['year']= data_weekday['iso'].apply(pd.Series).astype(int)[0]
data_weekday['week'] = data_weekday['iso'].apply(pd.Series).astype(int)[1]

del data_weekday['iso']
data_weekday_iso = data_weekday.groupby(['year','week']).mean()
del data_weekday_iso['week_num']

In [18]:
data_weekday_iso

comment_count
year week               
2012 1            1561.6
     2            2062.8
     3            1555.0
     4            1662.0
     5            1795.6
     6            1453.4
     7            1909.0
     8            1542.8
     9            1963.2
     10           1956.0
     11           2001.0
     12           2225.4
     13           2061.8
     14           2371.0
     15           1783.6
     16           2129.8
     17           2095.2
     18           1689.4
     19           1938.4
     20           1808.2
     21           2318.8
     22           2008.0
     23           1863.6
     24           1909.0
     25           1923.2
     26           1798.4
     27           1474.8
     28           1823.6
     29           1985.4
     30           2003.8
...                  ...
2013 24           1743.4
     25           1446.4
     26           1692.4
     27           1313.2
     28           1820.2
     29           1605.2
     30           1947.2
     31           2091.2
     32           1538.0
     33           1623.6
     34           2104.2
     35           2210.0
     36           1518.0
     37           1978.8
     38           1531.2
     39           1548.0
     40           1562.8
     41           1388.4
     42           1855.8
     43           1870.8
     44           1908.0
     45           1879.0
     46           2034.2
     47           1859.6
     48           2190.6
     49           2091.0
     50           2402.4
     51           1964.0
     52           1354.4
2014 1            1437.0

[105 rows x 1 columns]